# Rmax prediction of 6-helicenes from neural networks models

Linear and non-linear models have been built for regression of Rmax from a dataset of 6-helicenes with up to 4 halogens (F,Cl,Br,I) elements replacing the hydrogen atoms. 

The fundamental 6-helicene is made of 6 carbon rings with 16 bonding hydrogens. For the neural network model, each molecule is represented by a 16-component vector, each component associated to the position of one hydrogen. For example, a 6-helicene with a Fl atom in the position of the 4th hydrogen and a Br atom in the position of the 8th hydrogen atom is represented as:

$$(H,H,H,Cl,H,H,H,Br,H,H,H,H,H,H,H,H)$$


The models respect specular symmetry with respect to the middle of the molecule, which means that for the model:

$$(H,H,H,Cl,H,H,H,Br,H,H,H,H,H,H,H,H) = (H,H,H,H,H,H,H,H,Br,H,H,H,Cl,H,H,H)$$

In [ ]:
# Load libraries

import numpy as np
import tensorflow as tf
tf.__version__

2023-05-11 17:36:53.756080: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-11 17:36:53.872558: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-11 17:36:54.323899: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-11 17:36:54.323946: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

'2.11.0'

In [4]:
#from google.colab import drive
#drive.mount('/content/drive')
#!ls "/content/drive/MyDrive"
#!pwd
#working_dir = '/content/drive/MyDrive/Colab_Notebooks/Rmax_prediction_6Helicenes'
working_dir = '.'

In [5]:
# Load pre-trained model

# model = tf.keras.models.load_model('./Models/R_model_invariant_linear')
# model.summary()

linear_model = tf.keras.models.load_model(working_dir+'/Models/R_model_invariant_linear')
non_linear_model = tf.keras.models.load_model(working_dir+'/Models/R_model_invariant_non-linear_many-body_dropout')

#print(linear_model.summary())
#print(non_linear_model.summary())

2023-05-11 17:37:36.004640: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-11 17:37:36.068289: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2023-05-11 17:37:36.068508: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-05-11 17:37:36.069833: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neur

In [6]:
# Auxiliar tools to define input molecules of the model

#x_int = np.zeros(16)
n_pos = 16
x0_int = [0]*n_pos
x0_str =['H']*n_pos
#print('Fundamental 6-helicene:')
#print(x0_int)
#print(x0_str)

# dictionary to convert integer representation to string representation
int2str = {
  "0": "H",
  "1": "Fl",
  "2": "Cl",
  "3": "Br",
  "4": "I"
}

# dictionary to convert string representation to integer representation
str2int = {
  "H": "0",
  "Fl": "1",
  "Cl": "2",
  "Br": "3",
  "I": "4"
}

# function to convert integer representation to string representation
def int_to_str(x_int):
  x_str = [int2str[str(k)] for k in x_int]
  return x_str

# function to convert string representation to integer representation
def str_to_int(x_str):
  x_int = [int(str2int[c]) for c in x_str]
  return x_int

# One-hot encode input features
n_atom_types = 5 # there are 5 atom types: hydrogen and 4 halogens
n_features = n_atom_types*n_pos

# functions to define one-hot encoded vector being input of the model
def int_to_ohe(x_int):
  x_ohe = np.zeros(n_features) 
  for j in range(n_pos):
    atom_id = x_int[j]
    x_ohe[j*n_atom_types+atom_id] = 1.0
  return x_ohe

def str_to_ohe(x_str):
  #x_int = [int_to_str[str(k)] for k in x_str]
  x_int = str_to_int(x_str)
  print(x_int)
  x_ohe = np.zeros(n_features) 
  for j in range(n_pos):
    atom_id = x_int[j]
    x_ohe[j*n_atom_types+atom_id] = 1.0
  return x_ohe

# function to reshape the input as (1,n_features) and split it into multiple inputs
# The model architecture is designed to respect symmetries
def int_to_input(x_int):
  x_input = np.zeros((1,n_features))
  x_ohe = int_to_ohe(x_int)
  x_input[0,:] = x_ohe[:]
  x_input = np.split(x_input,n_pos,axis=1)
  return x_input

def str_to_input(x_str):
  x_input = np.zeros((1,n_features))
  x_int = str_to_int(x_str)
  x_ohe = int_to_ohe(x_int)
  x_input[0,:] = x_ohe[:]
  x_input = np.split(x_input,n_pos,axis=1)
  return x_input

""" # Examples
print()
x_int = x0_int
x_int[2] = 3
#x_str2 = [int2str[str(k)] for k in x_int]
x_str2 = int_to_str(x_int)
print(x_str2)
x_ohe2 = int_to_ohe(x_int)
print(x_ohe2)

print()
x_str = x0_str
x_str[2] = 'Br'
#x_str2 = [int2str[str(k)] for k in x_int]
x_int2 = str_to_int(x_str)
print(x_int2)
x_ohe2 = int_to_ohe(x_int2)
print(x_ohe2)

# Rmax calculation example (integer representation)
n_data = 1
x_input = np.zeros((n_data,n_features))
x = x0_int.copy()
x[2] = 3
x_ohe = int_to_ohe(x)
print(x)
print(x_ohe)
x_input[0,:] = x_ohe[:]
x_input = np.split(x_input,n_pos,axis=1)
#print(x_input)
Rmax = model.predict(x_input)
Rmax = float(Rmax)
print(Rmax) 

# Example of molecule with integer representation
x = x0_int.copy()
x[2] = 3
x_input = int_to_input(x)
print(float(model.predict(x_input))) """


" # Examples\nprint()\nx_int = x0_int\nx_int[2] = 3\n#x_str2 = [int2str[str(k)] for k in x_int]\nx_str2 = int_to_str(x_int)\nprint(x_str2)\nx_ohe2 = int_to_ohe(x_int)\nprint(x_ohe2)\n\nprint()\nx_str = x0_str\nx_str[2] = 'Br'\n#x_str2 = [int2str[str(k)] for k in x_int]\nx_int2 = str_to_int(x_str)\nprint(x_int2)\nx_ohe2 = int_to_ohe(x_int2)\nprint(x_ohe2)\n\n# Rmax calculation example (integer representation)\nn_data = 1\nx_input = np.zeros((n_data,n_features))\nx = x0_int.copy()\nx[2] = 3\nx_ohe = int_to_ohe(x)\nprint(x)\nprint(x_ohe)\nx_input[0,:] = x_ohe[:]\nx_input = np.split(x_input,n_pos,axis=1)\n#print(x_input)\nRmax = model.predict(x_input)\nRmax = float(Rmax)\nprint(Rmax) \n\n# Example of molecule with integer representation\nx = x0_int.copy()\nx[2] = 3\nx_input = int_to_input(x)\nprint(float(model.predict(x_input))) "

In [7]:
# Auxiliar function to evaluate Rmax with original units

R_mean = 591.451949
R_std = 85.381995

def calculate_Rmax(x,rep,mod):
    if(rep=='int'): # integer represenation of the input vector
        x_input = int_to_input(x)
    elif(rep=='str'): # string represenation of the input vector
        x_input = str_to_input(x)
    if(mod=='linear'):
        model = linear_model
    elif(mod=='non_linear'):
        model = non_linear_model
    Rmax = float( model.predict(x_input) )
    Rmax = Rmax * R_std + R_mean
    return Rmax

In [8]:
# Examples 

print('Fundamental 6-helicene:')
print(x0_int)
print(x0_str)

# Note: x[] indexes range from 0 to 15 for the 16 components, as usual in Python

# Example of molecule with integer representation: (0,1,2,3,4) for (H,F,Cl,Br,I)
x = x0_int.copy()
x[5] = 3
Rmax = calculate_Rmax(x,rep='int',mod='linear')
print()
print('Br in 5th position:')
print(x)
print(Rmax)

# Example of molecule with string representation
x = x0_str.copy()
x[5] = 'Br'
Rmax = calculate_Rmax(x,rep='str',mod='linear')
print()
print('Br in 5h position:')
print(x)
print(Rmax)

x = x0_str.copy()
x[10] = 'Br'
Rmax = calculate_Rmax(x,rep='str',mod='linear')
print()
print('Br in 10 position:')
print(x)
print(Rmax)

x = x0_str.copy()
x[10] = 'Br'
Rmax = calculate_Rmax(x,rep='str',mod='non_linear')
print()
print('Br in 10 position, non-linear model:')
print(x)
print(Rmax)



Fundamental 6-helicene:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H']
1/1 [==============================] - 0s 140ms/step

Br in 5th position:
[0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
647.629036339589
1/1 [==============================] - 0s 21ms/step

Br in 5h position:
['H', 'H', 'H', 'H', 'H', 'Br', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H']
647.629036339589
1/1 [==============================] - 0s 20ms/step

Br in 10 position:
['H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'Br', 'H', 'H', 'H', 'H', 'H']
647.629036339589
1/1 [==============================] - 0s 87ms/step

Br in 10 position, non-linear model:
['H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'Br', 'H', 'H', 'H', 'H', 'H']
615.1709877956217


In [24]:
# Loop over molecules up to 4 substituents


# This is only for 4 substituents!!!
n_data = 16*4*15*3*14*2*13
n_data = 16*4*15*3*14*2*13 + 16*4*15*3*14*2
x_data = np.zeros((n_data,n_pos))
Rmax_data = np.zeros(n_data)

# Define input vector with integer representation
i_data = 0
for ipos in range(15):
    for iat in range(1,5): # between 1 and 4
        for jpos in range(15):
            if(jpos!=ipos):
                for jat in range(1,5):
                    if(jat!=iat):
                        for kpos in range(15):
                            if (kpos!=ipos and kpos!=jpos):
                                for kat in range(1,5):
                                    if(kat!=iat and kat!=jat):
                                        for lpos in range(15):
                                            if(lpos!=ipos and lpos!=jpos and lpos!=kpos):
                                                for lat in range(1,5):
                                                    if(lat!=iat and lat!=jat and lat!=kat):
                                                        x_data[i_data,ipos] = iat
                                                        x_data[i_data,jpos] = jat
                                                        x_data[i_data,kpos] = kat
                                                        x_data[i_data,lpos] = lat
                                                        i_data = i_data + 1


# convert to one-hot encode vectors
print(x_data[4000,:])

def ohe(x_data):
  x_ohe = np.zeros((x_data.shape[0],n_features))
  for i in range(x_data.shape[0]):
    for j in range(n_pos):
        atom_id = int(x_data[i,j])
        x_ohe[i,j*n_atom_types+atom_id] = 1.0
  return x_ohe

x_ohe = ohe(x_data)
print(x_ohe[4000,:])


[1. 0. 0. 0. 0. 2. 3. 0. 0. 0. 0. 0. 4. 0. 0. 0.]
[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 1. 0. 0. 0. 0.]


In [10]:
### TEST
import numpy as np

n_data = 30000000
n_pos = 16
x_data = np.zeros((n_data,n_pos))
Rmax_data = np.zeros(n_data)

i_comb = 0
for ipos in range(15):
    for iat in range(5):
        #for jpos in range(15):
        for jpos in range(ipos+1,15):
            #if(jpos!=ipos):
                for jat in range(5):
                    #for kpos in range(15):
                    for kpos in range(jpos+1,15):
                        #if(kpos!=ipos and kpos!=jpos):
                            for kat in range(5):
                                #for lpos in range(15):
                                for lpos in range(kpos+1,15):    
                                    #if(lpos!=ipos and lpos!=jpos and lpos!=kpos):
                                        for lat in range(5):
                                            if( (iat+jat+kat+lat)>0 ):
                                                x_data[i_comb,ipos] = iat
                                                x_data[i_comb,jpos] = jat
                                                x_data[i_comb,kpos] = kat
                                                x_data[i_comb,lpos] = lat
                                                i_comb = i_comb + 1
print(i_comb)




851760


In [9]:
for ipos in range(5):
    for jpos in range(ipos+1,5):
        print(jpos)

1
2
3
4
2
3
4
3
4
4
